In [ ]:
from Bio import Entrez

Entrez.email = "zeiberg.d@northeastern.edu"

import pandas as pd

from easydict import EasyDict

In [ ]:
from tqdm.notebook import tqdm

import numpy as np
import pickle

In [ ]:
def getSequenceFromNPID(npid):
    "Return the protein sequence from "
    handle = Entrez.efetch(db="protein",id=npid, rettype="fasta", retmode="text")
    lines = handle.readlines()
    lines = [l.strip() for l in lines]
    return "".join(lines[1:])

In [ ]:
# df = pd.read_csv("/ssdata/hgmd/HGMD_PRO_2019_1_hg19.vcf",delimiter="\t",header=14)
df = pd.read_csv("/data/projects/processBio/hgmd/HGMD_PRO_2019_1_hg19.vcf",delimiter="\t",header=14)

In [ ]:
df

In [ ]:
def getInfo(row):
    d = EasyDict()
    for vals in row["INFO"].split(";"):
        k,v = vals.split("=")
        d[k] = v
    if "PROT" in d:
        d.pid, d.variant = d.PROT.split(":")
    elif "DB" in d:
        d.pid = ""
        d.variant=""
        rsid= d.DB.replace("rs","")
        
    else:
        return d
    try:
        _,d.variant = d.variant.split(".")
        d.reference,location,d.missense = d.variant[0],int(d.variant[1:-1]),d.variant[-1]
    except ValueError:
        return d
    d.loc = location - 1
    return d

# Original Stats

In [ ]:
allGenes = set()
genesWithAtleastOneVariant = set()
validInfo = []
for i,row in tqdm(df.iterrows()):
    info = getInfo(row)
    allGenes.add(info.GENE)
    if "missense" in info:
        genesWithAtleastOneVariant.add(info.GENE)
        validInfo.append(info)

# All missense variants

In [ ]:
variantDF = pd.DataFrame(validInfo)

In [ ]:
variantDF

Number of genes with at least one variant

In [ ]:
variantDF.pid.unique().shape

In [ ]:
sequences = EasyDict()

In [ ]:
for npid in tqdm(protein_ids):
    sequences[npid] = getSequenceFromNPID(npid)

In [ ]:
pickle.dump(sequences, open("/ssdata/hgmd/sequences.pkl","wb"))

In [ ]:
import pickle
from easydict import EasyDict

In [ ]:
# sequences = pickle.load(open("/ssdata/hgmd/sequences.pkl","rb"))
sequences = pickle.load(open("/data/projects/processBio/hgmd/sequences.pkl","rb"))

In [ ]:
seqs = []
errors = {"match":[]}
for idx,row in tqdm(variantDF.iterrows(),total=variantDF.shape[0]):
    s = sequences[row.pid]
    loc = row["loc"]
    variant = s[:loc] + row.missense + s[loc+1:]
    if loc < len(s) and s[loc] == row.reference:
        seqs.append(variant)
    else:
        errors["match"].append(idx)
        seqs.append(np.nan)

In [ ]:
variantDF = variantDF.assign(seq=seqs)

In [ ]:
variantDF.shape

pedja says DM only

In [ ]:
variantDF = variantDF[(variantDF.CLASS != "DM?") & (~variantDF.PROT.str.contains("\*"))]

In [ ]:
variantDF

In [ ]:
np.unique(variantDF.seq.isnull(),return_counts=True)

In [ ]:
variantDF.to_csv("/data/projects/processBio/hgmd/variant_df.csv")